#### General Steps to Follow

1. Importing Packages
2. Defining x_train, x_test, y_train, y_test
3. Building the Neural Network
4. Training the Neural Network
5. Model Evaluation

## 1) Importing Packages

In [63]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 2) Defining x_train, x_test, y_train, y_test

#### Loading the training and test data from "my data" folder

In [38]:
train_data = np.load("../my data/train_data.npy", allow_pickle = True)
test_data = np.load("../my data/test_data.npy"  , allow_pickle = True)

* x_train and x_test will contain the images.
* y_train and y_test will contain the label of each image:
  - 1 if it is a happy image.
  - 0 if it is a sad image.

In [39]:
x_train = train_data[:,0]
y_train = train_data[:,1]
x_test = test_data[:,0]
y_test = test_data[:,1]

#### Reshaping the input

* I created temp varaibles and deleted them to save memory because the data size is large

In [40]:
temp = x_train.tolist()
del x_train
x_train = np.array(temp)
del temp

temp = x_test.tolist()
del x_test
x_test = np.array(temp)
del temp

temp = y_train.tolist()
del y_train
y_train = tf.convert_to_tensor(temp)
del temp

temp = y_test.tolist()
del y_test
y_test = tf.convert_to_tensor(temp)
del temp

#### Checking the shapes

In [41]:
print("Shape of x_train    : ", x_train.shape)
print("Shape of y_train    : ", y_train.shape)

print("--------------------------------------------------")

print("Shape of x_test    : ", x_test.shape)
print("Shape of y_test    : ", y_test.shape)

Shape of x_train    :  (6897, 100, 100, 3)
Shape of y_train    :  (6897,)
--------------------------------------------------
Shape of x_test    :  (1702, 100, 100, 3)
Shape of y_test    :  (1702,)


### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 3) Building the Neural Network

In [78]:
#del model
model = Sequential(
    [
        Conv2D(32, (3,3), activation = 'relu', input_shape = (100,100,3)),
        BatchNormalization(),
        MaxPooling2D(),

        Conv2D(64, (3,3), activation = 'relu'),
        BatchNormalization(),
        MaxPooling2D(),

        Conv2D(32, (3,3), activation = 'relu'),
        BatchNormalization(),
        MaxPooling2D(),
        
        Flatten(),
        Dense(256, activation = 'relu'),
        Dense(1, activation = 'linear')
    ]
)

In [79]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 98, 98, 32)        896       
                                                                 
 batch_normalization_3 (Bat  (None, 98, 98, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 49, 49, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_20 (Conv2D)          (None, 47, 47, 64)        18496     
                                                                 
 batch_normalization_4 (Bat  (None, 47, 47, 64)        256       
 chNormalization)                                                
                                                      

### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 4) Training the Neural Network

In [80]:
model.compile(
    optimizer = Adam(learning_rate = 0.01),
    loss = BinaryCrossentropy(from_logits = True)
)

In [81]:
model.fit(x_train, y_train, epochs = 5, batch_size = 32)

Epoch 1/5
216/216 [==============================] - 39s 174ms/step - loss: 0.4084
Epoch 2/5
216/216 [==============================] - 37s 172ms/step - loss: 0.0818
Epoch 3/5
216/216 [==============================] - 37s 172ms/step - loss: 0.0638
Epoch 4/5
216/216 [==============================] - 39s 179ms/step - loss: 0.0550
Epoch 5/5
216/216 [==============================] - 38s 175ms/step - loss: 0.0639


### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 5) Model Evaluation

In [82]:
def model_eval(y, y_hat):
    m = y.shape[0]
    
    y_hat = y_hat.numpy()
    y = y.numpy()
    y_temp = np.zeros(m, dtype = "int32")
    for i in range(len(y_hat)):
        if(y_hat[i] >= 0.5):
            y_temp[i] = int(1)
            
    accuracy = 100*(np.sum(y == y_temp)/m)
    print("Accuracy =", accuracy)

#### Evaluation on training data

In [83]:
output1 = model.predict(x_train)

216/216 [==============================] - 9s 40ms/step


In [84]:
y_hat = tf.nn.sigmoid(output1)
y = y_train
model_eval(y, y_hat)
# del output1
# del y_hat

Accuracy = 98.2891112077715


#### Evaluation on test data

In [85]:
output2 = model.predict(x_test)

54/54 [==============================] - 2s 40ms/step


In [86]:
y_hat = tf.nn.sigmoid(output2)
y = y_test
model_eval(y, y_hat)

Accuracy = 97.64982373678026


### Saving the model

In [89]:
model.save("../my data/model.h5")

### Loading the model

In [ ]:
model = tf.keras.models.load_model("../my data/model.h5")